In [1]:
import cv2
import os
import shutil
import heapq
import numpy as np
from scipy import stats as sts
import math
import xlwt
import xlrd
import xlutils.copy
import pandas as pd
from xlrd import open_workbook
from skimage import filters
import skimage.io as io
from skimage.morphology import disk
import matplotlib
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from scipy.signal import convolve2d
from PIL import Image, ImageDraw, ImageFont
from skimage.feature import local_binary_pattern
from skimage import feature as ft
from sklearn.metrics.cluster import entropy
from sklearn import metrics
from scipy.signal import resample
from scipy.fftpack import dct

In [2]:
def read_img_file(path):
    with open(path, 'rb') as img_file:
        bytes = img_file.read()
        nparr = np.fromstring(bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        return img
    
# save data into excel sheet
def to_excel(data_list,file_path,col):
    rb = xlrd.open_workbook(file_path)
    wb = xlutils.copy.copy(rb)
    ws = wb.get_sheet(0)
    for i in range(len(data_list)):
        ws.write(i + 1, col, data_list[i])
    wb.save(file_path)
    
# read data and seperate them into neoplasm and non-neoplasm
def read_list(path,num,column_name):
    df = pd.read_excel(path)
    data = df[column_name]
    ca_list = data[num:]  #ca: neoplasm
    noca_list= data[:num] #noca: non-neoplasm
    ca_data,noca_data = [],[]
    for data in ca_list:
        ca_data.append(float(data))
    for data in noca_list:
        noca_data.append(float(data))
    return ca_data,noca_data

def grey2mask(img_label, thr=0.5):
    mask = img_label / 255
    mask[mask > thr] = 1
    mask[mask <= thr] = 0
    return mask

# read sheet
def read_list_1(path,column_name):
    df = pd.read_excel(path)
    data = df[column_name]
    _list = data[:]
    _data = []
    for data in _list:
        _data.append(data)
    return _data

# PCI
def get_dominant_colors(infile,num_col):
    image = Image.open(infile)
    image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    image[:,:,0] = cv2.bilateralFilter(image[:,:,0],5,200,200)
    image = cv2.cvtColor(image, cv2.COLOR_LAB2BGR)   
    image = Image.fromarray(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    small_image = image.resize((480, 480))
    result = small_image.convert("P", palette=Image.ADAPTIVE, colors = num_col)  
    palette = result.getpalette()
    color_counts = sorted(result.getcolors(), reverse=True)
    colors = []
    for i in range(num_col):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index * 3 : palette_index * 3 + 3]
        colors.append(tuple(dominant_color))
    return colors

# HSI entropy
def rgbtohsi_1(rgb_lwpImg):
    rows = int(rgb_lwpImg.shape[0])
    cols = int(rgb_lwpImg.shape[1])
    b, g, r = cv2.split(rgb_lwpImg)
    # scale to [0,1]
    b = b / 255.0
    g = g / 255.0
    r = r / 255.0
    S_img = np.zeros((rows, cols, 1), dtype=np.uint8)
    hsi_lwpImg = rgb_lwpImg.copy()
    H, S, I = cv2.split(hsi_lwpImg)
    for i in range(rows):
        for j in range(cols):
            min_RGB = min(min(b[i, j], g[i, j]), r[i, j])
            sum = b[i, j]+g[i, j]+r[i, j]
            if sum == 0:
                S = 0
            else:
                S = 1 - 3 * min_RGB/sum
            S_img[i, j] = S * 255
    return S_img

# Color moments
def color_comment(img):
    r,g,b = cv2.split(img)
    color_featrue = []
    r_mean = np.mean(r)
    g_mean = np.mean(g)
    b_mean = np.mean(b)
    r_std = np.std(r)
    g_std = np.std(g)
    b_std = np.std(b)
    r_offset = (np.mean(np.abs((r - r_mean)**3)))**(1./3)
    g_offset = (np.mean(np.abs((g - g_mean)**3)))**(1./3)
    b_offset = (np.mean(np.abs((b - b_mean)**3)))**(1./3)
    color_featrue.extend([r_mean,g_mean,b_mean,r_std,g_std,b_std,r_offset,g_offset,b_offset])
    return color_featrue

### Define input and output path

In [3]:
num_ca = 5 # Modify the number according to the actual situation 
num_noca = 5 # Modify the number according to the actual situation 

ca_path = r"~\data\imgs_feature_extraction_quantification\except_for_width_height_ratio\1"
noca_path = r"~\data\imgs_feature_extraction_quantification\except_for_width_height_ratio\0"

ca_excel_path = "file path for the sheet that stores the feature prediction results"
noca_excel_path = "file path for the sheet that stores the feature prediction results"

In [4]:
ca_result_list = []
for img_file in os.listdir(ca_path):
    ca_result_list.append(img_file)    
to_excle(ca_result_list,ca_ecxle_path,0) 

noca_result_list = []
for img_file in os.listdir(noca_path):
    noca_result_list.append(img_file)    
to_excle(noca_result_list,noca_ecxle_path,0) 

## PCI

In [1]:
result_list = []
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    color = get_dominant_colors(img_path,5)
    color_list = []
    for i in color:
        C_i = np.mean(i)
        if C_i != 0:
            color_list.append(C_i)
    std_color = np.std(color_list)
    std_color = std_color/100
    result_list.append(std_color)
to_excle(result_list,ca_ecxle_path,1) 

In [2]:
result_list = []
for file in os.listdir(noca_path):
    img_path = os.path.join(noca_path,file)
    color = get_dominant_colors(img_path,5)
    color_list = []
    for i in color:
        C_i = np.mean(i)
        if C_i != 0:
            color_list.append(C_i)
    std_color = np.std(color_list)
    std_color = std_color/100
    print(std_color)
    result_list.append(std_color)
to_excle(result_list,noca_ecxle_path,1) 

In [3]:
# identify the best cut-off value for classfying images into 1 and 0 

col_name = "PCI" 
ca_list = read_list_1(ca_ecxle_path,col_name)
noca_list = read_list_1(noca_ecxle_path,col_name)
for ps in range(0,10000,1):
    score = ps/1000
    count_ca_right = 0
    count_noca_right = 0
    for k_ca in ca_list:  
        if k_ca >= score:
            count_ca_right += 1
    for k_noca in noca_list:
        if k_noca < score:
            count_noca_right += 1
    tpr_ca = count_ca_right/num_ca
    tnr_noca = count_noca_right/num_noca
    if  tpr_ca > 0.5  and tnr_noca > 0.5:
        print('cut-off value:',score)
        print('sensitivity:',format(tpr_ca,'.4f'))
        print('specificity:',format(tnr_noca,'.4f'))

### HSI entropy

In [4]:
result_list = []
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    image_HSI = rgbtohsi_1(_img)
    image = image_HSI
    va = entropy(image)
    result_list.append(va)
to_excle(result_list,ca_ecxle_path,2)

In [5]:
result_list = []
for file in os.listdir(noca_path):
    img_path = os.path.join(noca_path,file)
    _img = read_img_file(img_path)
    image_HSI = rgbtohsi_1(_img)
    image = image_HSI
    va = entropy(image)
    result_list.append(va)
to_excle(result_list,noca_ecxle_path,2)

In [6]:
col_name = "HSI entroy" 
ca_list = read_list_1(ca_ecxle_path,col_name)
noca_list = read_list_1(noca_ecxle_path,col_name)
for ps in range(0,10000,1):
    score = ps/1000
    count_ca_right = 0
    count_noca_right = 0
    for k_ca in ca_list:  
        if k_ca >= score:
            count_ca_right += 1
    for k_noca in noca_list:
        if k_noca < score:
            count_noca_right += 1
    tpr_ca = count_ca_right/num_ca
    tnr_noca = count_noca_right/num_noca
    if  tpr_ca > 0.55  and tnr_noca > 0.55:
        print('cut-off value:',score)
        print('sensitivity:',format(tpr_ca,'.4f'))
        print('specificity:',format(tnr_noca,'.4f'))

## Texture

In [7]:
result_list = []
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    image = cv2.cvtColor(_img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image, 24, 3.5)
    V_rout = list(np.concatenate(lbp))
    va = np.mean(V_rout)/1000000 - 8
    print(va)
    result_list.append(va)
to_excle(result_list,ca_ecxle_path,3) 

In [8]:
result_list = []
for file in os.listdir(noca_path):
    img_path = os.path.join(noca_path,file)
    _img = read_img_file(img_path)
    image = cv2.cvtColor(_img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image, 24, 3.5)
    V_rout = list(np.concatenate(lbp))
    va = np.mean(V_rout)/1000000 - 8
    result_list.append(va)
to_excle(result_list,noca_ecxle_path,3) 

In [9]:
col_name = "Texture" 
ca_list = read_list_1(ca_ecxle_path,col_name)
noca_list = read_list_1(noca_ecxle_path,col_name)
for ps in range(0,10000,1):
    score = ps/1000
    count_ca_right = 0
    count_noca_right = 0
    for k_ca in ca_list:  
        if k_ca < score:
            count_ca_right += 1
    for k_noca in noca_list:
        if k_noca > score:
            count_noca_right += 1
    tpr_ca = count_ca_right/num_ca
    tnr_noca = count_noca_right/num_noca
    if  tpr_ca > 0.5  and tnr_noca > 0.5:
        print('cut-off value:',score)
        print('sensitivity:',format(tpr_ca,'.4f'))
        print('specificity:',format(tnr_noca,'.4f'))

## Histogram

In [10]:
result_list = []
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    image_HSV = cv2.cvtColor(_img, cv2.COLOR_BGR2XYZ)
    image = image_HSV[:,:,0]
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) 
    dst = cv2.filter2D(image, -1, kernel=kernel) 
    features = ft.hog(dst, orientations = 9, pixels_per_cell = (20,20),cells_per_block = (2,2), 
                  block_norm = 'L1',transform_sqrt = True,feature_vector = True,visualize = False)
    va = np.std(features)*100
    result_list.append(va)
to_excle(result_list,ca_ecxle_path,4) 

In [11]:
result_list = []
for file in os.listdir(noca_path):
    img_path = os.path.join(noca_path,file)
    _img = read_img_file(img_path)
    image_HSV = cv2.cvtColor(_img, cv2.COLOR_BGR2XYZ)
    image = image_HSV[:,:,0]
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) 
    dst = cv2.filter2D(image, -1, kernel=kernel) 
    features = ft.hog(dst, orientations = 9, pixels_per_cell = (20,20),cells_per_block = (2,2), 
                  block_norm = 'L1',transform_sqrt = True,feature_vector = True,visualize = False)
    va = np.std(features)*100
    result_list.append(va)
to_excle(result_list,noca_ecxle_path,4) 

In [12]:
col_name = "Histogram" 
ca_list = read_list_1(ca_ecxle_path,col_name)
noca_list = read_list_1(noca_ecxle_path,col_name)
for ps in range(0,10000,1):
    score = ps/1000
    count_ca_right = 0
    count_noca_right = 0
    for k_ca in ca_list:  
        if k_ca < score:
            count_ca_right += 1
    for k_noca in noca_list:
        if k_noca > score:
            count_noca_right += 1
    tpr_ca = count_ca_right/num_ca
    tnr_noca = count_noca_right/num_noca
    if  tpr_ca > 0.5  and tnr_noca > 0.5:
        print('cut-off value:',score)
        print('sensitivity:',format(tpr_ca,'.4f'))
        print('specificity:',format(tnr_noca,'.4f'))

## color_comment

In [13]:
result_list = []
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    features  = color_comment(_img)
    va = np.mean(features)/100
    result_list.append(va)
to_excle(result_list,ca_ecxle_path,5) 

In [14]:
result_list = []
for file in os.listdir(noca_path):
    img_path = os.path.join(noca_path,file)
    _img = read_img_file(img_path)
    features  = color_comment(_img)
    va = np.mean(features)/100
    result_list.append(va)
to_excle(result_list,noca_ecxle_path,5) 

In [15]:
col_name = "color_comment" 
ca_list = read_list_1(ca_ecxle_path,col_name)
noca_list = read_list_1(noca_ecxle_path,col_name)
for ps in range(0,10000,1):
    score = ps/1000
    count_ca_right = 0
    count_noca_right = 0
    for k_ca in ca_list:  
        if k_ca < score:
            count_ca_right += 1
    for k_noca in noca_list:
        if k_noca > score:
            count_noca_right += 1
    tpr_ca = count_ca_right/num_ca
    tnr_noca = count_noca_right/num_noca
    if  tpr_ca > 0.5  and tnr_noca > 0.5:
        print('cut-off value:',score)
        print('sensitivity:',format(tpr_ca,'.4f'))
        print('specificity:',format(tnr_noca,'.4f'))

### Save the output

In [4]:
ca_result_list = []
for img_file in os.listdir(ca_path):
    ca_result_list.append(img_file)    
to_excel(ca_result_list,ca_excel_path,0) 

In [5]:
result_list = []
derta = N
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    color = get_dominant_colors(img_path,5)
    color_list = []
    for i in color:
        C_i = np.mean(i)
        if C_i != 0:
            color_list.append(C_i)
    std_color = sts.std(color_list)
    std_color = std_color/100
    if std_color >= derta:
        result_list.append(1)
    else:
        result_list.append(0)
to_excel(result_list,ca_excel_path,1)

In [6]:
result_list = []
derta = N
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    image_HSI = rgbtohsi_1(_img)
    image = image_HSI
    va = entropy(image)
    if va >= derta:
        result_list.append(1)
    else:
        result_list.append(0)
to_excel(result_list,ca_excel_path,2)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  after removing the cwd from sys.path.


In [7]:
result_list = []
derta = N
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    image = cv2.cvtColor(_img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image, 24, 3.5)
    V_rout = list(np.concatenate(lbp))
    va = np.mean(V_rout)/1000000 - 8
    if va <= derta:
        result_list.append(1)
    else:
        result_list.append(0)
to_excel(result_list,ca_excel_path,3) 

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  after removing the cwd from sys.path.


In [8]:
result_list = []
derta = N
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    image_HSV = cv2.cvtColor(_img, cv2.COLOR_BGR2XYZ)
    image = image_HSV[:,:,0]
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) 
    dst = cv2.filter2D(image, -1, kernel=kernel) 
    features = ft.hog(dst, orientations = 9, pixels_per_cell = (20,20),cells_per_block = (2,2), 
                  block_norm = 'L1',transform_sqrt = True,feature_vector = True,visualize = False)
    va = np.std(features)*100
    if va <= derta:
        result_list.append(1)
    else:
        result_list.append(0)
to_excel(result_list,ca_excel_path,4) 

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  after removing the cwd from sys.path.


In [9]:
result_list = []
derta = N
for file in os.listdir(ca_path):
    img_path = os.path.join(ca_path,file)
    _img = read_img_file(img_path)
    features  = color_comment(_img)
    va = np.mean(features)/100
    if va <= derta:
        result_list.append(1)
    else:
        result_list.append(0)
to_excel(result_list,ca_excel_path,5) 

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  after removing the cwd from sys.path.
